In [1]:
from dotenv import load_dotenv, dotenv_values
import os
import openai
from langchain_openai import AzureChatOpenAI, AzureOpenAI, AzureOpenAIEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient, SearchIndexingBufferedSender
from azure.search.documents.indexes import SearchIndexClient
from langchain_openai import AzureChatOpenAI, AzureOpenAI, AzureOpenAIEmbeddings
from langchain.embeddings import OpenAIEmbeddings
from openai import AzureOpenAI
from azure.search.documents.models import (
    QueryAnswerType,
    QueryCaptionType,
    QueryCaptionResult,
    QueryAnswerResult,
    SemanticErrorMode,
    SemanticErrorReason,
    SemanticSearchResultsType,
    QueryType,
    VectorizedQuery,
    VectorQuery,
    VectorFilterMode
)
from azure.search.documents.indexes.models import (
    ExhaustiveKnnAlgorithmConfiguration,
    ExhaustiveKnnParameters,
    SearchIndex,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    SearchableField,
    SemanticConfiguration,
    SemanticPrioritizedFields,
    SemanticField,
    SemanticSearch,    
    HnswAlgorithmConfiguration,
    HnswParameters,
    VectorSearch,
    VectorSearchAlgorithmConfiguration,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile  
)

load_dotenv()


True

In [2]:
#Define Azure OpenAI credentials
api_key = os.getenv("AZURE_OPENAI_KEY")
api_version = "2023-12-01-preview"
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
engine = os.getenv("EMBEDDING_DEPLOYMENT_NAME")

#Define Azure AI Search credentials
service_endpoint = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("AZURE_SEARCH_INDEX_NAME")
key = os.getenv("AZURE_SEARCH_ADMIN_KEY")
credential = AzureKeyCredential(key)

In [3]:
#create embedding-client using AzureOpenAI credentials
client = AzureOpenAI(api_key=api_key,
    api_version="2023-12-01-preview",
    azure_endpoint =azure_endpoint
    )

### 1) embed a given query

In [4]:
def query_embedding(client, input, engine):
    client=client
    response= client.embeddings.create(input=input,model=engine)
    query_embedding = response.data[0].embedding 
    return query_embedding

In [5]:
query="Mental Health"
query_vector=query_embedding(client, query, engine)
print(query_vector)

[0.006083027459681034, 0.014423673041164875, 0.03348800539970398, -0.033939529210329056, -0.028069721534848213, 0.022952454164624214, -0.006017180159687996, -0.017320949584245682, 0.007280819118022919, -0.017283322289586067, -0.005506080109626055, 0.012554866261780262, -0.0036090537905693054, -0.013207067735493183, -0.0003027795464731753, 0.019528398290276527, 0.03747646510601044, -0.019453145563602448, 0.0028016415890306234, -0.014122657477855682, -0.03075377829372883, 0.01594129391014576, -0.013871810398995876, -0.005418283864855766, -0.01133826095610857, 0.004270661622285843, 0.00969521701335907, -0.032334111630916595, -0.007920477539300919, -0.02716667391359806, 0.025197530165314674, -0.012843339703977108, -0.011068601161241531, -0.027141589671373367, -0.013620964251458645, 0.006434212438762188, 0.0026636761613190174, -0.001779442303813994, -5.315789621818112e-06, -0.014925366267561913, 0.005763197783380747, -0.0011256736470386386, -0.002052237745374441, -0.0009618395124562085, -0.

### 2) perform the search

#### 2.1) Normal Vector search

In [6]:
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
vector_query = VectorizedQuery(vector=query_vector, k_nearest_neighbors=3, fields="embedding")

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    select=["line", "filename"],
    top=3
)

for result in results:
    print(f"Score: {result['@search.score']}")
    print(f"Captions: {result['@search.captions']}")
    print(f"Highlights: {result['@search.highlights']}")
    print(f"Content: {result['line']}\n")
    print(f"Filename: {result['filename']}\n")
    print("######################################")

Score: 0.8069528
Captions: None
Highlights: None
Content:  Teamwork: We believe that by working together, we can achieve greater success. 5. Respect: We treat all our employees, c ustomers, and partners with respect and dignity. 6. Excellence: We strive to exceed expectations and provide excellent service. 7. Accountability: We take responsibility for our actions and hold ourselves and others accountable for their performance. 8. Co mmunity: We are committed to making a positive impact in the communities in which we work and live

Filename: employee_handbook.pdf

######################################
Score: 0.8041429
Captions: None
Highlights: None
Content:  Purpose The purpose of this program is to promote a safe and healthy work environment by preventing violence, threats, and abuse in the workplace. It is also intended to provide a safe, secure and protected environment for our employees, customers, and visitors. Definition of Workplace Violence Workplace violence is any act of phy

#### 2.2) Exhaustive KNN search

In [7]:
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
vector_query = VectorizedQuery(vector=query_vector, k_nearest_neighbors=3, fields="embedding", exhaustive=True)

results = search_client.search(
    search_text=None,
    vector_queries=[vector_query],
    select=["line", "filename"],
    top=3
)

for result in results:
    print(f"Score: {result['@search.score']}")
    print(f"Content: {result['line']}\n")
    print(f"Filename: {result['filename']}\n")
    print("######################################")


Score: 0.8069523
Content:  Teamwork: We believe that by working together, we can achieve greater success. 5. Respect: We treat all our employees, c ustomers, and partners with respect and dignity. 6. Excellence: We strive to exceed expectations and provide excellent service. 7. Accountability: We take responsibility for our actions and hold ourselves and others accountable for their performance. 8. Co mmunity: We are committed to making a positive impact in the communities in which we work and live

Filename: employee_handbook.pdf

######################################
Score: 0.80414295
Content:  Purpose The purpose of this program is to promote a safe and healthy work environment by preventing violence, threats, and abuse in the workplace. It is also intended to provide a safe, secure and protected environment for our employees, customers, and visitors. Definition of Workplace Violence Workplace violence is any act of physical aggression, intimidation, or threat of physical harm towa

#### 2.3) Hybrid Search - if available in Azure region

In [8]:
search_client = SearchClient(service_endpoint, index_name, AzureKeyCredential(key))
vector_query = VectorizedQuery(vector=query_vector, k_nearest_neighbors=3, fields="embedding")

results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    select=["line", "filename"],
    query_type=QueryType.SEMANTIC,
    semantic_configuration_name='my-semantic-config',
    query_caption=QueryCaptionType.EXTRACTIVE,
    query_answer=QueryAnswerType.EXTRACTIVE,
    top=3
)


Semantic ranking of answers

In [9]:
for result in results:
    print(f"Reranker Score: {result['@search.reranker_score']}")
    print(f"Content: {result['line']}\n") 

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")


Reranker Score: 1.2143075466156006
Content:  Purpose The purpose of this program is to promote a safe and healthy work environment by preventing violence, threats, and abuse in the workplace. It is also intended to provide a safe, secure and protected environment for our employees, customers, and visitors. Definition of Workplace Violence Workplace violence is any act of physical aggression, intimidation, or threat of physical harm toward another individual in the workplace

Caption: employee_handbook.pdf.  Purpose The purpose of this program is to promote a safe and healthy work environment by preventing violence, threats, and abuse in the workplace. It is also intended to provide a safe, secure and protected environment for our employees, customers, and visitors. Definition of Workplace Violence Workplace violence is any act of<em> physical</em> aggression, intimidation, or threat of<em> physical</em> harm toward another individual in the workplace.

Reranker Score: 0.916360259056091